In [1]:
%pip install stable-baselines3[extra]
%pip install gymnasium
%pip install pyboy
%load_ext tensorboard

import tensorflow
import gymnasium

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


2024-05-08 15:45:27.772228: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-08 15:45:28.064192: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 15:45:28.064243: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 15:45:28.066027: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-08 15:45:28.254080: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-08 15:45:28.256037: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [5]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from pyboy import PyBoy

actions = ['a', 'b', 'left', 'right', 'up', 'down', 'start', 'select']
screen_shape = (144, 160, 4)
game_area_observation_space = spaces.Box(low=0, high=255, shape=screen_shape, dtype=np.uint8)

class PokemonRedEnv(gym.Env):    
    def __init__(self, debug=False):
        super(PokemonRedEnv, self).__init__()

        if debug: 
            self.pyboy = PyBoy('pokemon_red.gb')
        else:
            self.pyboy = PyBoy('pokemon_red.gb', window="null")

        
        self.debug = debug

        if not self.debug:
            self.pyboy.set_emulation_speed(0)

        self.action_space = spaces.Discrete(len(actions))
        self.observation_space = game_area_observation_space

        print("init")
        self.reset()

    def reset(self, **kwargs):
        print("reset")
        self.steps = 0
        with open('pokemon_red.gb.state', 'rb') as f:
            self.pyboy.load_state(f)

        return self._observation(), {}

    def step(self, action):
        self.pyboy.button(actions[action])
        self.pyboy.tick(1, False)
        
        self.steps += 1

        terminate = False
        reward = 0

        followed_oak = self._read_bit(0xD74B, 7)
        if followed_oak:
            terminate = True
            reward = 1
        elif self.steps > 300:
          print("Boo")
          terminate = True
 

        return self._observation(), reward, terminate, False, {}
    
    def _read_bit(self, address, bit):
        return bin(256 + self.pyboy.memory[address])[-bit-1] == '1'

    def _observation(self):
        return self.pyboy.screen.ndarray





In [6]:
from genericpath import exists
from pathlib import Path
import uuid

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from stable_baselines3 import PPO

from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common import env_checker

session_path = 'Sessions/'
tensorboard_path = 'Tensorboard/'

def train():
    ep_length = 2048 * 8
    sess_path = Path(f'{session_path}/session_{str(uuid.uuid4())[:8]}')

    env = DummyVecEnv([lambda: PokemonRedEnv()])
    checkpoint_callback = CheckpointCallback(save_freq=ep_length, save_path=sess_path,
                                         name_prefix='trade')

    learn_steps = 40
    file_name = f'{session_path}/session_1/tradd1'

    if exists(file_name + '.zip'):
        print('\nloading checkpoint')
        model = PPO.load(file_name, env=env,  tensorboard_log=tensorboard_path)
        model.n_steps = ep_length
        model.n_envs = 1
        model.rollout_buffer.buffer_size = ep_length
        model.rollout_buffer.n_envs = 1
        model.rollout_buffer.reset()
    else:
        model = PPO('CnnPolicy', env, verbose=1, n_steps=ep_length, batch_size=512, n_epochs=1, tensorboard_log=tensorboard_path)

    #model.save("drive/MyDrive/Stocks/v1")


    for i in range(learn_steps):
        print(f"Step {i}")
        model.learn(total_timesteps=(ep_length)*1*1000, callback=checkpoint_callback,tb_log_name="first_run", reset_num_timesteps=False)


In [7]:
train()

    

init
reset
Using cpu device
Wrapping the env in a VecTransposeImage.
Step 0
reset
Logging to Tensorboard/first_run_0
Boo
reset
Boo
reset
Boo
reset
